In [2]:
import tensorflow as tf
from inception_resnet_v1 import *
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)  

In [4]:
model = InceptionResNetV1()

model.load_weights('facenet_weights.h5')
 
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [5]:
def l2_normalize(x):   #L2정규화
 return x / np.sqrt(np.sum(np.multiply(x, x)))

In [6]:
def findEuclideanDistance(source_representation, test_representation): #유클리드 거리
 euclidean_distance = source_representation - test_representation
 euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
 euclidean_distance = np.sqrt(euclidean_distance)
 return euclidean_distance

In [7]:
def preprocess_image(image_path):  #이미지 전처리
    img = load_img(image_path, target_size=(160, 160))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    img = preprocess_input(img)
    return img

In [10]:
img1_representation = l2_normalize(model.predict(preprocess_image("dataset/facenet/minsu.jpg"))[0,:])
img2_representation = l2_normalize(model.predict(preprocess_image("dataset/facenet/lim.jpg"))[0,:])

#img1_representation = model.predict(preprocess_image("dataset/facenet/minsu.jpg"))[0,:]
#img2_representation = model.predict(preprocess_image("dataset/facenet/lim.jpg"))[0,:]

euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)

In [11]:
threshold = 0.35
if euclidean_distance < threshold:
 print("같은 사람",euclidean_distance)
else:
 print("다른 사람", euclidean_distance)

다른 사람 5.494402


In [16]:
#코사인 유사도
def findCosineSimilarity(source_representation, test_representation):
 a = np.matmul(np.transpose(source_representation), test_representation)
 b = np.sum(np.multiply(source_representation, source_representation))
 c = np.sum(np.multiply(test_representation, test_representation))
 return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
 
img1_representation = model.predict(preprocess_image("dataset/facenet/minsu.jpg"))[0,:]
img2_representation = model.predict(preprocess_image("dataset/facenet/minsu3.jpg"))[0,:]
 
cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
threshold = 0.07
if cosine_similarity < threshold:
 print("같은 사람", cosine_similarity)
else:
 print("다른 사람", cosine_similarity)

같은 사람 0.02416139841079712
